# Accelerating End-to-End Data Science Workflows # 

## 04 - การถดถอยโลจิสติก (Logistic Regression) ##

**สารบัญ**
<br>
สมุดบันทึก (notebook) นี้ใช้การถดถอยโลจิสติกที่เร่งความเร็วด้วย GPU เพื่อทำนายความเสี่ยงของการติดเชื้อ โดยอิงจากคุณลักษณะของสมาชิกประชากรของเรา สมุดบันทึกนี้ครอบคลุมหัวข้อด้านล่าง:

1.  [สภาพแวดล้อม (Environment)](#Environment)
2.  [โหลดข้อมูล (Load Data)](#Load-Data)
3.  [การถดถอยโลจิสติก (Logistic Regression)](#Logistic-Regression)
    * [การดูผลการถดถอย (Viewing the Regression)](#Viewing-the-Regression)
    * [การประมาณความน่าจะเป็นของการติดเชื้อ (Estimate Probability of Infection)](#Estimate-Probability-of-Infection)
4.  [ความสามารถในการอธิบายของโมเดล (Model Explainability)](#Model-Explainability)
    * [แสดงให้เห็นว่าความชุกของการติดเชื้อเกี่ยวข้องกับอายุ (Show Infection Prevalence is Related to Age)](#Show-Infection-Prevalence-is-Related-to-Age)
    * [แบบฝึกหัดที่ 1 - แสดงให้เห็นว่าความชุกของการติดเชื้อเกี่ยวข้องกับเพศ (Exercise #1 - Show Infection Prevalence is Related to Sex)](#Exercise-#1---Show-Infection-Prevalence-is-Related-to-Sex)
5.  [การทำนายด้วยข้อมูลการฝึกอบรมและการทดสอบที่แยกกัน (Making Predictions with Separate Training and Testing Data)](#Making-Predictions-with-Separate-Training-and-Test-Data)
    * [แบบฝึกหัดที่ 2 - การปรับโมเดลการถดถอยโลจิสติกโดยใช้ข้อมูลการฝึกอบรม (Exercise #2 - Fit Logistic Regression Model Using Training Data)](#Exercise-#2---Fit-Logistic-Regression-Model-Using-Training-Data)
    * [ใช้ข้อมูลทดสอบเพื่อตรวจสอบความถูกต้องของโมเดล (Use Test Data to Validate Model)](#Use-Test-Data-to-Validate-Model)

## สภาพเเวดล้อม ##

In [ ]:
import cudf
import cuml

import cupy as cp

## โหลดข้อมูล

In [ ]:
gdf = cudf.read_csv('./data/clean_uk_pop_full.csv', usecols=['age', 'sex', 'infected'])

In [ ]:
gdf.dtypes

In [ ]:
gdf.shape

In [ ]:
gdf.head()

## Logistic Regression ##
Logistic regression (การถดถอยโลจิสติก) สามารถใช้เพื่อประมาณ **ความน่าจะเป็นของผลลัพธ์** โดยขึ้นอยู่กับตัวแปรนำเข้าบางตัว (ที่ถือว่าเป็นอิสระต่อกัน) ในกรณีของเรา เราต้องการประมาณ **ความเสี่ยงในการติดเชื้อ** โดยพิจารณาจากอายุและเพศของประชากร

ด้านล่างนี้ เราจะทำการฝึกโมเดล logistic regression ก่อนอื่นเราจะสร้าง instance ของ cuML logistic regression ที่ชื่อว่า `logreg` เมธอด `logreg.fit` ใช้ 2 อาร์กิวเมนต์: ตัวแปรอิสระของโมเดล *X* และตัวแปรตาม *y* ทำการ fit โมเดล `logreg` โดยใช้คอลัมน์ `age` และ `sex` จาก `gdf` เป็น *X* และคอลัมน์ `infected` เป็น *y*

In [ ]:
logreg = cuml.LogisticRegression()
logreg.fit(gdf[['age', 'sex']], gdf['infected'])

### การดูผลการถดถอย (Viewing the Regression) ###

หลังจากที่เราได้ทำการปรับโมเดล (fitting the model) แล้ว เราสามารถใช้ `logreg.predict` เพื่อประเมินว่าบุคคลใดมีโอกาสติดเชื้อมากกว่า 50% หรือไม่ อย่างไรก็ตาม เนื่องจากไวรัสนี้มีความชุกต่ำในประชากร (ประมาณ 1-2% ในชุดข้อมูลนี้) ความน่าจะเป็นรายบุคคลของการติดเชื้อจึงต่ำกว่า 50% มาก และโมเดลควรจะทำนายได้อย่างถูกต้องว่าไม่มีใครมีแนวโน้มที่จะติดเชื้อเป็นการส่วนตัว

อย่างไรก็ตาม เรายังสามารถเข้าถึง **สัมประสิทธิ์ของโมเดล (model coefficients)** ได้ที่ `logreg.coef_` รวมถึง **ค่าจุดตัดแกน (intercept)** ที่ `logreg.intercept_` ค่าทั้งสองนี้เป็น cuDF Series

ด้านล่างนี้เราจะมาดูค่าเหล่านี้ จะสังเกตได้ว่าการเปลี่ยนเพศจาก 0 เป็น 1 มีผลผ่านสัมประสิทธิ์เช่นเดียวกับการเปลี่ยนอายุประมาณ 48 ปี

In [ ]:
type(logreg.coef_)

In [ ]:
type(logreg.intercept_)

In [ ]:
logreg_coef = logreg.coef_
logreg_int = logreg.intercept_

print("Coefficients: [age, sex]")
print([logreg_coef[0], logreg_coef[1]])

print("Intercept:")
print(logreg_int[0])

### ประมาณความน่าจะเป็นของการติดเชื้อ ###
เช่นเดียวกับการถดถอยโลจิสติกส์ทั้งหมด ค่าสัมประสิทธิ์ช่วยให้เราสามารถคำนวณโลจิตสำหรับแต่ละรายการได้ จากนั้นเราสามารถคำนวณความเสี่ยงของการติดเชื้อเป็นเปอร์เซ็นต์โดยประมาณได้

**หมายเหตุ**: เมื่อจำได้ว่า 1 หมายถึง 'ติดเชื้อ' เราจะกำหนดความน่าจะเป็นของคลาสนั้นให้กับคอลัมน์ใหม่ในดาต้าเฟรมต้นฉบับ

In [ ]:
class_probs = logreg.predict_proba(gdf[['age', 'sex']])
class_probs

In [ ]:
gdf['risk'] = class_probs[1]

Looking at the original records with their new estimated risks, we can see how estimated risk varies across individuals.

In [ ]:
gdf.take(cp.random.choice(gdf.shape[0], size=5, replace=False))

## Model Explainability ##
**Model explainability** (ความสามารถในการอธิบายโมเดล) หมายถึง ความสามารถในการทำความเข้าใจและอธิบายการตัดสินใจและเหตุผลเบื้องหลังการคาดการณ์จากโมเดลแมชชีนเลิร์นนิง สามารถทำได้โดยการตรวจสอบว่าตัวแปรคุณสมบัติ (feature variables) มีความสัมพันธ์กับตัวแปรเป้าหมาย (target variable) อย่างไร

### แสดงให้เห็นว่าความชุกของการติดเชื้อสัมพันธ์กับอายุ ###

ค่าสัมประสิทธิ์ที่เป็นบวกของอายุบ่งชี้ว่าไวรัสแพร่หลายในกลุ่มผู้สูงอายุมากกว่า แม้จะควบคุมตัวแปรเพศแล้วก็ตาม

สำหรับแบบฝึกหัดนี้ ให้แสดงความสัมพันธ์ระหว่างความชุกของการติดเชื้อกับอายุ โดยการพิมพ์ค่าเฉลี่ยของ `infected` สำหรับสมาชิกที่อายุมากที่สุดและอายุน้อยที่สุดของประชากร เมื่อจัดกลุ่มตามอายุ:


In [ ]:
# %load solutions/risk_by_age
age_groups = gdf[['age', 'infected']].groupby(['age'])
print(age_groups.mean().head())
print(age_groups.mean().tail())


### แบบฝึกหัดที่ 1 - แสดงให้เห็นว่าความชุกของการติดเชื้อเกี่ยวข้องกับเพศ ###
ในทำนองเดียวกัน ค่าสัมประสิทธิ์ที่เป็นบวกของเพศ (sex) บ่งชี้ว่าไวรัสแพร่หลายในคนที่มีเพศ = `1` (เพศหญิง) มากกว่า แม้ว่าจะควบคุมปัจจัยด้านอายุแล้วก็ตาม

**คำแนะนำ**: <br>
* แก้ไขเฉพาะส่วนที่ `<FIXME>` และรันเซลล์ด้านล่างเพื่อแสดงให้เห็นว่าความชุกของการติดเชื้อมีความสัมพันธ์กับเพศ โดยการพิมพ์ค่าเฉลี่ยของ `infected` สำหรับประชากรเมื่อจัดกลุ่มตามเพศ








In [ ]:
sex_groups = gdf[<<<<FIXME>>>>].groupby(<<<<FIXME>>>>)
sex_groups.mean()

Click ... for solution. 

## การสร้างการคาดการณ์ด้วยข้อมูลการฝึก (Training Data) และข้อมูลการทดสอบ (Test Data) ที่แยกกัน ##
การประมวลผลโดยทั่วไปจะประกอบด้วยการ **ฝึกฝนโมเดล (training the model)** ด้วยชุดข้อมูลสำหรับฝึก (training set) จากนั้นจึงใช้ชุดข้อมูลสำหรับทดสอบ (test set) เพื่อ **ประเมินประสิทธิภาพ (evaluate its performance)** ซึ่งจะช่วยให้การประเมินผลนั้นสมจริงมากขึ้น ว่าโมเดลจะทำงานได้ดีเพียงใดกับข้อมูลใหม่ที่ไม่เคยเห็นมาก่อนในการใช้งานจริง การทดสอบด้วยชุดข้อมูลที่แยกต่างหากช่วยให้คุณสามารถตรวจจับได้ว่าโมเดลของคุณกำลังเกิด **ภาวะโอเวอร์ฟิตติ้ง (overfitting)** กับข้อมูลสำหรับฝึกหรือไม่ ภาวะโอเวอร์ฟิตติ้งเกิดขึ้นเมื่อโมเดลทำงานได้ดีกับข้อมูลสำหรับฝึก แต่ทำงานได้ไม่ดีกับข้อมูลใหม่ ในหลายๆ กรณี คุณไม่สามารถเข้าถึงข้อมูลใหม่จริงๆ ได้ ดังนั้นการแบ่งข้อมูลที่มีอยู่จึงเป็นการจำลองสถานการณ์นี้

cuML มีเมธอดง่ายๆ สำหรับการสร้างชุดข้อมูลฝึก/ทดสอบที่จับคู่กัน:

In [ ]:
X_train, X_test, y_train, y_test  = cuml.train_test_split(gdf[['age', 'sex']], gdf['infected'], train_size=0.9)

### แบบฝึกหัดที่ 2 - การปรับใช้โมเดล Logistic Regression ด้วยข้อมูลสำหรับฝึก (Training Data) ###

**คำแนะนำ**:
* รันเซลล์ด้านล่างเพื่อสร้างโมเดล logistic regression ใหม่ชื่อ `logreg`
* แก้ไขเฉพาะส่วนที่ระบุว่า `<FIXME>` และรันเซลล์ด้านล่างเพื่อปรับใช้โมเดลใหม่ด้วยข้อมูลสำหรับฝึก *X* และ *y* ที่เพิ่งสร้างไป

In [ ]:
logreg = cuml.LogisticRegression()

In [ ]:
logreg.fit(<<<<FIXME>>>>, <<<<FIXME>>>>)

Click ... for solution. 

### ใช้ข้อมูลทดสอบเพื่อตรวจสอบความถูกต้องของโมเดล (Use Test Data to Validate Model) ###
ตอนนี้เราสามารถใช้ขั้นตอนเดียวกันกับข้างต้น เพื่อทำนายความเสี่ยงของการติดเชื้อโดยใช้ข้อมูลทดสอบได้แล้ว:

In [ ]:
y_test_pred = logreg.predict_proba(X_test, convert_dtype=True)[1]
y_test_pred.index = X_test.index
y_test_pred

ดังที่เราเห็นก่อนหน้านี้ มีคนจำนวนน้อยมากที่ติดเชื้อในประชากร แม้แต่ในกลุ่มที่มีความเสี่ยงสูงสุด เพื่อเป็นวิธีง่ายๆ ในการตรวจสอบโมเดลของเรา เราได้แบ่งชุดข้อมูลทดสอบออกเป็นกลุ่มที่มีความเสี่ยงที่คาดการณ์ไว้สูงกว่าค่าเฉลี่ย และกลุ่มที่มีความเสี่ยงที่คาดการณ์ไว้ต่ำกว่าค่าเฉลี่ย จากนั้นสังเกตว่า **ความชุกของการติดเชื้อมีความสัมพันธ์ใกล้ชิดกับความเสี่ยงที่คาดการณ์ไว้เหล่านั้น**

In [ ]:
test_results = cudf.DataFrame()
test_results['age'] = X_test['age']
test_results['sex'] = X_test['sex']
test_results['infected'] = y_test
test_results['predicted_risk'] = y_test_pred

test_results['high_risk'] = test_results['predicted_risk'] > test_results['predicted_risk'].mean()

risk_groups = test_results.groupby('high_risk')
risk_groups.mean()

สุดท้ายนี้ ภายในไม่กี่มิลลิวินาที เราสามารถทำการวิเคราะห์สองระดับตาม **เพศ** และ **อายุ** ได้:

In [ ]:
%%time
s_groups = test_results[['sex', 'age', 'infected', 'predicted_risk']].groupby(['sex', 'age'])
s_groups.mean()

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**ทำได้ดีมาก!** ไปยัง [สมุดบันทึกถัดไป](3-05_knn.ipynb) กันเลย